In [1]:
! pip install deap
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 6.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import scipy.stats as stats
import statsmodels.stats.multicomp as mc
import array
# import pairwise_tukeyhsd


from deap import base
from deap import creator
from deap import tools
from deap import algorithms


In [3]:
def generateES(icls,scls,size,imin,imax,smin,smax):
  ind = icls(random.uniform(imin, imax) for _ in range(size))
  ind.strategy = scls(random.uniform(smin,smax) for _ in range(size))
  return ind

In [4]:
# @title Criando as entidades
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

In [5]:
toolbox = base.Toolbox()
IND_SIZE = 30
MIN_VALUE = -32.768
MAX_VALUE = 32.768
MIN_STRATEGY = 0.2
MAX_STRATEGY = 3

toolbox.register("individual", generateES, creator.Individual, creator.Strategy,
    IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRATEGY, MAX_STRATEGY)
# Attribute generator




# gerador de indivíduos considera 100 posições binárias na lista
# toolbox.register("individual",generateES,creator.Individual,creator.Strategy, D, -32.768,32.768,0.01,1.0)
# população é uma lista de indivíduos
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# gerador gera um atributo inteiro binário
# toolbox.register("attr_bool", random.uniform, -30, 30)


In [6]:
def ackley_function(individual):
    a = 20
    b = 0.2
    c = 2*np.pi

    first_term = -a * np.exp(-b * np.sqrt(np.mean(np.square(individual))))
    second_term = -np.exp(np.mean(np.cos(c * np.array(individual))))

    return first_term + second_term + a + np.exp(1),


In [7]:
def mutGau(individual,sigma,indpb):
  for i in range(len(individual)):
    if random.random() < indpb:
      individual[i] = random.gauss(individual[i],sigma*individual[i])
  return individual,


def checkStrategy(minstrategy):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrappper
    return decorator



toolbox.register("mate", tools.cxESBlend, alpha=0.1)
toolbox.register("mutate", tools.mutESLogNormal, c = 1.0, indpb=0.4)
toolbox.register("select", tools.selTournament, tournsize=30)
toolbox.register("evaluate", ackley_function)

toolbox.decorate("mate", checkStrategy(MIN_STRATEGY))
toolbox.decorate("mutate", checkStrategy(MIN_STRATEGY))


In [8]:

def mains(population,geracao):
    MU, LAMBDA = population, geracao
    pop = toolbox.population(n=MU)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA,
        cxpb=0.6, mutpb=0.3, ngen=200000//LAMBDA, stats=stats, halloffame=hof)

    # # Obtendo o melhor fitness encontrado nesta execução
    # best_fitness = hof[0].fitness.values[0] if len(hof) > 0 else None
    # print(f"Melhor fitness encontrado: {best_fitness}")

    # print(f"Terminado um experimento com tamPop={population} e nGer={geracao}")
    return pop, logbook, hof

In [9]:
# # Parâmetros para rodar os testes
# tamPops = [20, 30, 40]
# nGers = [100, 200, 300]
# nTestes = 30

# # Dicionário para armazenar os resultados dos testes
# results = {f"mu_{mu}_lambda_{lambdas}": [] for mu in tamPops for lambdas in nGers}

# # Rodando os testes e armazenando os melhores fitness
# for mu in tamPops:
#     for lambdas in nGers:
#         for _ in range(nTestes):
#             best_fitness = mains(mu, lambdas)
#             if best_fitness is not None:
#                 results[f"mu_{mu}_lambda_{lambdas}"].append(best_fitness)

# @title Execucoes
mus = [20, 30, 40]
lambdas = [100, 200, 300]
N = 30

results = []
for mu in mus:
  for lambda_ in lambdas:
    result = []
    for _ in range(N):
      pop, logbook, hof = mains(mu, lambda_)
      result.append(hof[0].fitness.values[0])
    results.append(result)



A saída de streaming foi truncada nas últimas 5000 linhas.
343	271   	3.0711 	0          	3.0711 	3.0711 
344	265   	3.0711 	0          	3.0711 	3.0711 
345	274   	3.0711 	0          	3.0711 	3.0711 
346	265   	3.0711 	0          	3.0711 	3.0711 
347	271   	3.0711 	0          	3.0711 	3.0711 
348	272   	3.0711 	0          	3.0711 	3.0711 
349	263   	3.0711 	0          	3.0711 	3.0711 
350	277   	3.0711 	0          	3.0711 	3.0711 
351	277   	3.0711 	0          	3.0711 	3.0711 
352	269   	3.0711 	0          	3.0711 	3.0711 
353	266   	3.0711 	0          	3.0711 	3.0711 
354	267   	3.0711 	0          	3.0711 	3.0711 
355	268   	3.0711 	0          	3.0711 	3.0711 
356	272   	3.0711 	0          	3.0711 	3.0711 
357	272   	3.0711 	0          	3.0711 	3.0711 
358	275   	3.0711 	0          	3.0711 	3.0711 
359	272   	3.0711 	0          	3.0711 	3.0711 
360	265   	3.0711 	0          	3.0711 	3.0711 
361	273   	3.0711 	0          	3.0711 	3.0711 
362	266   	3.0711 	0          	3.0711 	3.0711 
3

In [10]:
results

[[3.433746609844953,
  2.1308828354269846,
  2.243086680566567,
  2.6101141185164463,
  2.2215261685332135,
  2.2577765029896892,
  2.792599232602886,
  2.4273862706934897,
  2.508046733017576,
  3.9327795005733965,
  5.410834455251429,
  2.3266398296780406,
  2.605827617414214,
  2.6304325700422457,
  2.8563984184122657,
  2.236058380653191,
  3.1208431744313327,
  4.586546845194416,
  2.899277688508701,
  3.314195541687998,
  2.7439079900576506,
  2.7413756214316902,
  3.299839392187693,
  2.582469580048969,
  3.1141709309117513,
  3.694950596362798,
  2.154511173262055,
  2.6112693027148635,
  3.3109752947243547,
  2.405819792306549],
 [2.4967216745097933,
  2.462774229333292,
  2.1299614678371914,
  1.3394836666177707,
  2.7786813815353146,
  3.3093247869695834,
  2.5831097731799244,
  2.320945653597072,
  2.1332670347375857,
  2.1945524855919207,
  2.944543155608383,
  2.6465113219472447,
  2.442159467032998,
  3.189433905366791,
  2.8789349876320043,
  3.2177520536447877,
  4.199

In [11]:
# @title Testes estatisticos (ANOVA)


from scipy.stats import f_oneway
grau_confianca = 0.05

f, p_valor = f_oneway(*results)

print(f"Valor F: {f}; p-valor: {p_valor}")

if p_valor < grau_confianca:
  print("Há diferença considerável! Conduza o teste de tukey")
else:
  print("Não há diferença considerável. Não há necessidade de conduzir o teste de tukey")

Valor F: 3.6332280412451254; p-valor: 0.0004993477840212183
Há diferença considerável! Conduza o teste de tukey


In [12]:
# @title Testes estatisticos (Tukey)
from scipy.stats import tukey_hsd

tukey_results = tukey_hsd(*results)

print(tukey_results)

Tukey's HSD Pairwise Group Comparisons (95.0% Confidence Interval)
Comparison  Statistic  p-value  Lower CI  Upper CI
 (0 - 1)      0.208     0.912    -0.271     0.688
 (0 - 2)      0.328     0.451    -0.152     0.807
 (0 - 3)      0.199     0.932    -0.281     0.678
 (0 - 4)      0.563     0.009     0.084     1.043
 (0 - 5)      0.512     0.026     0.032     0.991
 (0 - 6)      0.224     0.873    -0.256     0.703
 (0 - 7)      0.449     0.087    -0.031     0.928
 (0 - 8)      0.630     0.002     0.150     1.109
 (1 - 0)     -0.208     0.912    -0.688     0.271
 (1 - 2)      0.119     0.997    -0.360     0.599
 (1 - 3)     -0.010     1.000    -0.489     0.470
 (1 - 4)      0.355     0.337    -0.125     0.834
 (1 - 5)      0.304     0.559    -0.176     0.783
 (1 - 6)      0.015     1.000    -0.464     0.495
 (1 - 7)      0.240     0.821    -0.239     0.720
 (1 - 8)      0.422     0.136    -0.058     0.901
 (2 - 0)     -0.328     0.451    -0.807     0.152
 (2 - 1)     -0.119     0.997   